# 21기 KNN 정규세션 과제

**데이터:** [blackfriday | Kaggle](https://www.kaggle.com/llopesolivei/blackfriday)

---

## 0. 데이터 불러오기

In [1]:
import pandas as pd
import os

data_dir = os.getcwd()
df = pd.read_csv(os.path.join(data_dir, "blackfriday.csv"), index_col = 0)
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1001088,P00046042,F,0-17,10,A,3,0,5,17.0,NaN,2010
1,1004493,P00347742,F,0-17,10,A,1,0,7,NaN,NaN,4483
2,1005302,P00048942,F,0-17,10,A,1,0,1,4.0,NaN,7696
3,1001348,P00145242,F,0-17,10,A,3,0,2,4.0,NaN,16429
4,1001348,P00106742,F,0-17,10,A,3,0,3,5.0,NaN,5780


## 1. Preprocssing / EDA

In [2]:
df.shape

(4998, 12)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4998 entries, 0 to 4997
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   User_ID                     4998 non-null   int64  
 1   Product_ID                  4998 non-null   object 
 2   Gender                      4998 non-null   object 
 3   Age                         4998 non-null   object 
 4   Occupation                  4998 non-null   int64  
 5   City_Category               4998 non-null   object 
 6   Stay_In_Current_City_Years  4998 non-null   object 
 7   Marital_Status              4998 non-null   int64  
 8   Product_Category_1          4998 non-null   int64  
 9   Product_Category_2          3465 non-null   float64
 10  Product_Category_3          1544 non-null   float64
 11  Purchase                    4998 non-null   int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 507.6+ KB


In [4]:
df.describe()

,User_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
count,4.998000e+03,4998.000000,4998.000000,4998.000000,3465.000000,1544.000000,4998.000000
mean,1.003015e+06,8.036815,0.421369,5.320128,9.773737,12.766839,9234.655462
std,1.709910e+03,6.442697,0.493828,3.912281,5.101867,4.023591,4999.225081
min,1.000003e+06,0.000000,0.000000,1.000000,2.000000,3.000000,13.000000
25%,1.001560e+06,2.000000,0.000000,1.000000,5.000000,9.000000,5497.750000
50%,1.003040e+06,7.000000,0.000000,5.000000,9.000000,14.000000,8049.000000
75%,1.004417e+06,14.000000,1.000000,8.000000,15.000000,16.000000,12039.000000
max,1.006037e+06,20.000000,1.000000,20.000000,18.000000,18.000000,23913.000000


In [5]:
df.isnull().sum()

User_ID                          0
Product_ID                       0
Gender                           0
Age                              0
Occupation                       0
City_Category                    0
Stay_In_Current_City_Years       0
Marital_Status                   0
Product_Category_1               0
Product_Category_2            1533
Product_Category_3            3454
Purchase                         0
dtype: int64

In [6]:
# 'User_ID' 열 제거 (예측에 불필요)
df = df.drop('User_ID', axis=1)

# 'Product_ID' 열 제거 (예측에 불필요)
df = df.drop('Product_ID', axis=1)

# 'Product_Category_2' 열 제거 (결측치 존재)
df = df.drop('Product_Category_2', axis=1)

# 'Product_Category_3' 열 제거 (결측치 존재)
df = df.drop('Product_Category_3', axis=1)

# 결측치가 있는 행 제거
df = df.dropna()

df = df.reset_index(drop=True)

df.shape

(4998, 8)

In [7]:
# 각 칼럼에 대한 값 확인

for column in df.columns:
    print(f'---- {column} ----')
    print(df[column].value_counts())
    print('\n')

---- Gender ----
M    3765
F    1233
Name: Gender, dtype: int64


---- Age ----
26-35    1996
36-45    1000
18-25     906
46-50     414
51-55     350
55+       195
0-17      137
Name: Age, dtype: int64


---- Occupation ----
4     706
0     572
7     535
1     460
17    339
12    305
20    285
2     251
14    245
16    244
6     186
3     160
15    133
10    122
5      98
11     95
19     73
13     65
18     56
9      50
8      18
Name: Occupation, dtype: int64


---- City_Category ----
B    2102
C    1555
A    1341
Name: City_Category, dtype: int64


---- Stay_In_Current_City_Years ----
1     1758
2      915
3      908
4+     742
0      675
Name: Stay_In_Current_City_Years, dtype: int64


---- Marital_Status ----
0    2892
1    2106
Name: Marital_Status, dtype: int64


---- Product_Category_1 ----
5     1407
1     1285
8      995
2      227
11     206
3      193
6      193
4      107
16      78
15      57
13      43
10      41
7       40
12      31
20      31
19      20
14      19
18 

In [8]:
catCol = df.select_dtypes(include=['object']).columns.tolist()
numCol = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [9]:
numCol.remove('Marital_Status')

In [10]:
from sklearn.preprocessing import StandardScaler

df = pd.get_dummies(df, columns=catCol)

scaler = StandardScaler()

for col in numCol:
    df[[col]] = scaler.fit_transform(df[[col]])

df

,Occupation,Marital_Status,Product_Category_1,Purchase,Gender_F,Gender_M,Age_0-17,Age_18-25,Age_26-35,Age_36-45,...,Age_51-55,Age_55+,City_Category_A,City_Category_B,City_Category_C,Stay_In_Current_City_Years_0,Stay_In_Current_City_Years_1,Stay_In_Current_City_Years_2,Stay_In_Current_City_Years_3,Stay_In_Current_City_Years_4+
0,0.304745,0,-0.081835,-1.445300,1,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
1,0.304745,0,0.429427,-0.950574,1,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
2,0.304745,0,-1.104358,-0.307810,1,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,0.304745,0,-0.848727,1.439236,1,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
4,0.304745,0,-0.593097,-0.691107,1,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,-0.160945,1,-1.104358,0.493395,0,1,0,0,0,0,...,0,1,0,0,1,0,1,0,0,0
4994,-1.092325,1,0.685058,-1.438298,0,1,0,0,0,0,...,0,1,0,0,1,0,1,0,0,0
4995,0.770435,1,-1.104358,-0.946172,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,1,0
4996,-1.247555,1,1.707582,-1.564730,0,1,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0


In [11]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['Marital_Status'])
y = df['Marital_Status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


## 2. KNN 구현 & 파라미터 튜닝

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [13]:
y_train

3714    0
910     0
56      0
3005    1
2497    1
       ..
1381    0
3568    0
132     0
1643    0
4822    1
Name: Marital_Status, Length: 3998, dtype: int64

In [15]:
knn = KNeighborsClassifier()

# Grid search를 위한 하이퍼파라미터 설정
param_grid = {
    'n_neighbors': [5, 10, 15],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [30, 40, 50],
    'p': [1, 2],
    'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski', 'wminkowski', 'seuclidean', 'mahalanobis']
}

# Grid search 진행
grid_search = GridSearchCV(knn, param_grid, cv=3, verbose=3)
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터와 그 때의 점수 출력
print("best params: ", grid_search.best_params_)
print("best score: ", grid_search.best_score_)

# 최적의 하이퍼파라미터로 학습된 모델로 테스트 데이터 예측
knn_pred = grid_search.predict(X_test)


Fitting 3 folds for each of 1008 candidates, totalling 3024 fits
[CV 1/3] END algorithm=auto, leaf_size=30, metric=euclidean, n_neighbors=5, p=1, weights=uniform;, score=0.626 total time=   0.0s
[CV 2/3] END algorithm=auto, leaf_size=30, metric=euclidean, n_neighbors=5, p=1, weights=uniform;, score=0.635 total time=   0.0s
[CV 3/3] END algorithm=auto, leaf_size=30, metric=euclidean, n_neighbors=5, p=1, weights=uniform;, score=0.619 total time=   0.0s
[CV 1/3] END algorithm=auto, leaf_size=30, metric=euclidean, n_neighbors=5, p=1, weights=distance;, score=0.633 total time=   0.0s
[CV 2/3] END algorithm=auto, leaf_size=30, metric=euclidean, n_neighbors=5, p=1, weights=distance;, score=0.662 total time=   0.0s
[CV 3/3] END algorithm=auto, leaf_size=30, metric=euclidean, n_neighbors=5, p=1, weights=distance;, score=0.642 total time=   0.0s
[CV 1/3] END algorithm=auto, leaf_size=30, metric=euclidean, n_neighbors=5, p=2, weights=uniform;, score=0.626 total time=   0.0s
[CV 2/3] END algorithm

[CV 3/3] END algorithm=auto, leaf_size=30, metric=manhattan, n_neighbors=15, p=1, weights=uniform;, score=0.637 total time=   0.0s
[CV 1/3] END algorithm=auto, leaf_size=30, metric=manhattan, n_neighbors=15, p=1, weights=distance;, score=0.653 total time=   0.0s
[CV 2/3] END algorithm=auto, leaf_size=30, metric=manhattan, n_neighbors=15, p=1, weights=distance;, score=0.659 total time=   0.0s
[CV 3/3] END algorithm=auto, leaf_size=30, metric=manhattan, n_neighbors=15, p=1, weights=distance;, score=0.679 total time=   0.0s
[CV 1/3] END algorithm=auto, leaf_size=30, metric=manhattan, n_neighbors=15, p=2, weights=uniform;, score=0.636 total time=   0.0s
[CV 2/3] END algorithm=auto, leaf_size=30, metric=manhattan, n_neighbors=15, p=2, weights=uniform;, score=0.644 total time=   0.0s
[CV 3/3] END algorithm=auto, leaf_size=30, metric=manhattan, n_neighbors=15, p=2, weights=uniform;, score=0.637 total time=   0.0s
[CV 1/3] END algorithm=auto, leaf_size=30, metric=manhattan, n_neighbors=15, p=2

[CV 3/3] END algorithm=auto, leaf_size=30, metric=minkowski, n_neighbors=10, p=1, weights=distance;, score=0.675 total time=   0.0s
[CV 1/3] END algorithm=auto, leaf_size=30, metric=minkowski, n_neighbors=10, p=2, weights=uniform;, score=0.623 total time=   0.0s
[CV 2/3] END algorithm=auto, leaf_size=30, metric=minkowski, n_neighbors=10, p=2, weights=uniform;, score=0.623 total time=   0.0s
[CV 3/3] END algorithm=auto, leaf_size=30, metric=minkowski, n_neighbors=10, p=2, weights=uniform;, score=0.627 total time=   0.0s
[CV 1/3] END algorithm=auto, leaf_size=30, metric=minkowski, n_neighbors=10, p=2, weights=distance;, score=0.638 total time=   0.0s
[CV 2/3] END algorithm=auto, leaf_size=30, metric=minkowski, n_neighbors=10, p=2, weights=distance;, score=0.659 total time=   0.0s
[CV 3/3] END algorithm=auto, leaf_size=30, metric=minkowski, n_neighbors=10, p=2, weights=distance;, score=0.656 total time=   0.0s
[CV 1/3] END algorithm=auto, leaf_size=30, metric=minkowski, n_neighbors=15, p=

D:\ProgramFiles\Anaconda3-2020.02-Windows-x86_64.exe\lib\site-packages\sklearn\model_selection\_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "D:\ProgramFiles\Anaconda3-2020.02-Windows-x86_64.exe\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "D:\ProgramFiles\Anaconda3-2020.02-Windows-x86_64.exe\lib\site-packages\sklearn\metrics\_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "D:\ProgramFiles\Anaconda3-2020.02-Windows-x86_64.exe\lib\site-packages\sklearn\base.py", line 651, in score
    return accuracy_score(y, self.predict(X), sample_weight=sample_weight)
  File "D:\ProgramFiles\Anaconda3-2020.02-Windows-x86_64.exe\lib\site-packages\sklearn\neighbors\_classification.py", line 214, in predict
    neigh_dist, neigh_ind = s

[CV 2/3] END algorithm=auto, leaf_size=30, metric=seuclidean, n_neighbors=5, p=2, weights=distance;, score=nan total time=   0.0s
[CV 3/3] END algorithm=auto, leaf_size=30, metric=seuclidean, n_neighbors=5, p=2, weights=distance;, score=nan total time=   0.0s
[CV 1/3] END algorithm=auto, leaf_size=30, metric=seuclidean, n_neighbors=10, p=1, weights=uniform;, score=nan total time=   0.0s
[CV 2/3] END algorithm=auto, leaf_size=30, metric=seuclidean, n_neighbors=10, p=1, weights=uniform;, score=nan total time=   0.0s
[CV 3/3] END algorithm=auto, leaf_size=30, metric=seuclidean, n_neighbors=10, p=1, weights=uniform;, score=nan total time=   0.0s
[CV 1/3] END algorithm=auto, leaf_size=30, metric=seuclidean, n_neighbors=10, p=1, weights=distance;, score=nan total time=   0.0s
[CV 2/3] END algorithm=auto, leaf_size=30, metric=seuclidean, n_neighbors=10, p=1, weights=distance;, score=nan total time=   0.0s
[CV 3/3] END algorithm=auto, leaf_size=30, metric=seuclidean, n_neighbors=10, p=1, weigh

...

[CV 1/3] END algorithm=brute, leaf_size=50, metric=minkowski, n_neighbors=5, p=2, weights=distance;, score=0.633 total time=   0.0s
[CV 2/3] END algorithm=brute, leaf_size=50, metric=minkowski, n_neighbors=5, p=2, weights=distance;, score=0.662 total time=   0.0s
[CV 3/3] END algorithm=brute, leaf_size=50, metric=minkowski, n_neighbors=5, p=2, weights=distance;, score=0.642 total time=   0.0s
[CV 1/3] END algorithm=brute, leaf_size=50, metric=minkowski, n_neighbors=10, p=1, weights=uniform;, score=0.633 total time=   0.0s
[CV 2/3] END algorithm=brute, leaf_size=50, metric=minkowski, n_neighbors=10, p=1, weights=uniform;, score=0.638 total time=   0.0s
[CV 3/3] END algorithm=brute, leaf_size=50, metric=minkowski, n_neighbors=10, p=1, weights=uniform;, score=0.639 total time=   0.0s
[CV 1/3] END algorithm=brute, leaf_size=50, metric=minkowski, n_neighbors=10, p=1, weights=distance;, score=0.651 total time=   0.0s
[CV 2/3] END algorithm=brute, leaf_size=50, metric=minkowski, n_neighbors=1

D:\ProgramFiles\Anaconda3-2020.02-Windows-x86_64.exe\lib\site-packages\sklearn\model_selection\_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "D:\ProgramFiles\Anaconda3-2020.02-Windows-x86_64.exe\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "D:\ProgramFiles\Anaconda3-2020.02-Windows-x86_64.exe\lib\site-packages\sklearn\metrics\_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "D:\ProgramFiles\Anaconda3-2020.02-Windows-x86_64.exe\lib\site-packages\sklearn\base.py", line 651, in score
    return accuracy_score(y, self.predict(X), sample_weight=sample_weight)
  File "D:\ProgramFiles\Anaconda3-2020.02-Windows-x86_64.exe\lib\site-packages\sklearn\neighbors\_classification.py", line 214, in predict
    neigh_dist, neigh_ind = s

D:\ProgramFiles\Anaconda3-2020.02-Windows-x86_64.exe\lib\site-packages\sklearn\model_selection\_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "D:\ProgramFiles\Anaconda3-2020.02-Windows-x86_64.exe\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "D:\ProgramFiles\Anaconda3-2020.02-Windows-x86_64.exe\lib\site-packages\sklearn\metrics\_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "D:\ProgramFiles\Anaconda3-2020.02-Windows-x86_64.exe\lib\site-packages\sklearn\base.py", line 651, in score
    return accuracy_score(y, self.predict(X), sample_weight=sample_weight)
  File "D:\ProgramFiles\Anaconda3-2020.02-Windows-x86_64.exe\lib\site-packages\sklearn\neighbors\_classification.py", line 214, in predict
    neigh_dist, neigh_ind = s

[CV 3/3] END algorithm=brute, leaf_size=50, metric=seuclidean, n_neighbors=10, p=1, weights=distance;, score=nan total time=   0.0s
[CV 1/3] END algorithm=brute, leaf_size=50, metric=seuclidean, n_neighbors=10, p=2, weights=uniform;, score=nan total time=   0.0s
[CV 2/3] END algorithm=brute, leaf_size=50, metric=seuclidean, n_neighbors=10, p=2, weights=uniform;, score=nan total time=   0.0s
[CV 3/3] END algorithm=brute, leaf_size=50, metric=seuclidean, n_neighbors=10, p=2, weights=uniform;, score=nan total time=   0.0s
[CV 1/3] END algorithm=brute, leaf_size=50, metric=seuclidean, n_neighbors=10, p=2, weights=distance;, score=nan total time=   0.0s
[CV 2/3] END algorithm=brute, leaf_size=50, metric=seuclidean, n_neighbors=10, p=2, weights=distance;, score=nan total time=   0.0s
[CV 3/3] END algorithm=brute, leaf_size=50, metric=seuclidean, n_neighbors=10, p=2, weights=distance;, score=nan total time=   0.0s
[CV 1/3] END algorithm=brute, leaf_size=50, metric=seuclidean, n_neighbors=15, 

D:\ProgramFiles\Anaconda3-2020.02-Windows-x86_64.exe\lib\site-packages\sklearn\model_selection\_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "D:\ProgramFiles\Anaconda3-2020.02-Windows-x86_64.exe\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "D:\ProgramFiles\Anaconda3-2020.02-Windows-x86_64.exe\lib\site-packages\sklearn\metrics\_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "D:\ProgramFiles\Anaconda3-2020.02-Windows-x86_64.exe\lib\site-packages\sklearn\base.py", line 651, in score
    return accuracy_score(y, self.predict(X), sample_weight=sample_weight)
  File "D:\ProgramFiles\Anaconda3-2020.02-Windows-x86_64.exe\lib\site-packages\sklearn\neighbors\_classification.py", line 214, in predict
    neigh_dist, neigh_ind = s

## 3. Evaluation

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 성능 평가 지표 계산
accuracy = accuracy_score(y_test, knn_pred)
precision = precision_score(y_test, knn_pred)
recall = recall_score(y_test, knn_pred)
f1 = f1_score(y_test, knn_pred)

# 성능 평가 지표 출력
print('Accuracy: ', accuracy)
print('Precision: ', precision)
print('Recall: ', recall)
print('F1 Score: ', f1)


Accuracy:  0.676
Precision:  0.6577017114914425
Recall:  0.5938189845474614
F1 Score:  0.6241299303944314
